## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np
import sys

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial
value = int(sys.argv[1])
X = np.random.rand(value)

# Para imprimir los pimeros valores del array
# print(X[0:12])

# Utilizando las operaciones mágicas de ipython
tiempo = %timeit -r 2 -o -q reduc_operation(X)
print("Time taken by reduction operation using a function:", tiempo)
print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()
tiempo = %timeit -r 2 -o -q np.sum(X)
print("Time taken by reduction operation using numpy.sum():", tiempo)
print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()
tiempo= %timeit -r 2 -o -q X.sum()
print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)
print("Now, the result using numpy.ndarray.sum():", X.sum())

Time taken by reduction operation using a function: 45.7 ms ± 742 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24959.88879934843

Time taken by reduction operation using numpy.sum(): 95.6 µs ± 107 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 24959.88879934858 
 
Time taken by reduction operation using numpy.ndarray.sum(): 72.5 µs ± 116 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 24959.88879934858


## APARTADO 3.2 a) Librería multiprocessing

In [2]:
from multiprocessing import Pool

if __name__ == "__main__":
    # Primera prueba: utilizando un único proceso
    with Pool(processes=1) as pool:
        time_single = %timeit -r 2 -o -q pool.map(reduc_operation, [X])
        print("Tiempo con un único proceso:", time_single)
        
    # Segunda prueba: utilizando dos procesos
    with Pool(processes=2) as pool:
        # Dividimos el array en dos partes
        split_array_2 = [X[:int(value/2)], X[int(value/2):]]
        time_two = %timeit -r 2 -o -q pool.map(reduc_operation, split_array_2)
        print("Tiempo con dos procesos:", time_two)
                                          
    # Tercera prueba: utilizando cuatro procesos
    with Pool(processes=4) as pool:
        # Dividimos el array en cuatro partes
        split_array_4 = [X[:int(value/4)], X[int(value/4):int(value/2)], X[int(value/2):int(3*value/4)], X[int(3*value/4):]]
        time_four = %timeit -r 2 -o -q pool.map(reduc_operation, split_array_4)
        print("Tiempo con cuatro procesos:", time_four)


Tiempo con un único proceso: 51.1 ms ± 232 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Tiempo con dos procesos: 30.4 ms ± 1.06 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)
Tiempo con cuatro procesos: 29.8 ms ± 13.2 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)


## APARTADO 3.2 b) Librería Numba

In [3]:
from numba import njit, prange

# Primera implementación de la función con njit
@njit
def compute_sum_njit(array):
    """Sum the elements of the array from index 0 to value."""
    total = 0
    for idx in range(array.size):
        total += array[idx]
    return total

# Medimos el tiempo para la versión secuencial
tiempo_secuencial = %timeit -r 2 -o -q compute_sum_njit(X)
print("Tiempo con njit (sin paralelismo):", tiempo_secuencial)

# Segunda implementación de la función con njit y paralelismo
@njit(parallel=True)
def compute_sum_njit_parallel(array):
    """Sum the elements of the array using parallel execution."""
    total = 0
    for idx in prange(array.size):
        total += array[idx]
    return total

# Medimos el tiempo para la versión paralela
tiempo_parallel = %timeit -r 2 -o -q compute_sum_njit_parallel(X)
print("Tiempo con njit y paralelismo:", tiempo_parallel)


Tiempo con njit (sin paralelismo): 236 µs ± 17.1 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Tiempo con njit y paralelismo: 72.1 ms ± 64.3 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


## APARTADO 3.2 d) Comentarios

OUTPUT:

Ejecutando el notebook con 50000000 elementos
Time taken by reduction operation using a function: 45 s ± 70.2 ms per loop (mea
n ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25004901.471310124
Time taken by reduction operation using numpy.sum(): 59.7 ms ± 323 µs per loop (
mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 25004901.47130974 
Time taken by reduction operation using numpy.ndarray.sum(): 60.5 ms ± 480 µs pe
r loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.ndarray.sum(): 25004901.47130974

Tiempo con un único proceso: 49.4 s ± 81 ms per loop (mean ± std. dev. of 2 runs
, 1 loop each)
Tiempo con dos procesos: 25.7 s ± 6.19 ms per loop (mean ± std. dev. of 2 runs, 
1 loop each)
Tiempo con cuatro procesos: 13.3 s ± 7.22 ms per loop (mean ± std. dev. of 2 run
s, 1 loop each)

Tiempo con njit (sin paralelismo): 208 ms ± 2 ms per loop (mean ± std. dev. of 2
 runs, 1 loop each)
Tiempo con njit y paralelismo: 14.1 ms ± 2 ms per loop (mean ± std. dev. of 2 ru
ns, 1 loop each)

Los resultados muestran que el uso de multiprocessing permite reducir el tiempo de ejecución conforme se aumenta el número de procesos, aproximadamente a la mitad al duplicar los procesos, lo cual es esperado debido a la naturaleza del paralelismo en datos. Sin embargo, hay una ligera sobrecarga inicial al manejar los procesos.

Por otro lado, Numba demuestra ser significativamente más rápido que multiprocessing, tanto en su versión secuencial como paralela. La versión paralela logra una pequeña mejora respecto a la secuencial.

En conclusión, Numba es la opción más eficiente para este problema, mientras que multiprocessing ofrece flexibilidad al aprovechar múltiples núcleos.